In [1]:
#
import os
import sys
import warnings
from pathlib import Path
import dataclasses
from typing import Dict, Union

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# Stable Baseline3
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

# AI4Finance
from meta.config import TIME_ZONE_PARIS

#
import seaborn as sns
from copy import deepcopy
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyfolio
from pyfolio import timeseries
import tqdm
import yfinance as yf
import pyfolio as pf

#
from rl.plot.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from common.utils import now_time
from configuration.settings import ProjectDir, ExperimentDir
from rl.data.CompanyInfo import CompanyInfo
from rl.envs.StockTradingEnv import StockTradingEnv
from rl.experiments._1_same_bigger_data_fundamental.train import (
    Program,
    CustomDRLAgent,
    get_dataset,
    get_env_kwargs,
    dataset_name,
    base_cols,
    data_cols,
    ratios_cols,
    algorithm_name,
)

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
def ignore_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)


ignore_warnings()

In [3]:
def main():
    program = Program(
        prj_dir=ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing")),
        exp_dir=ExperimentDir(Path(os.getcwd())),
        DEBUG=False,
    )
    program.dataset = get_dataset(
        pd.read_csv(program.exp_dir.out.datasets.joinpath(f"{dataset_name}.csv"), index_col=0), purpose="test"
    )
    program.exp_dir.check_and_create_dirs()
    return program


program = main()

In [4]:
print(base_cols)
print(data_cols)
print(ratios_cols)

['date', 'tic']
['open', 'high', 'low', 'close', 'volume']
['operatingProfitMargin', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity', 'currentRatio', 'quickRatio', 'cashRatio', 'inventoryTurnover', 'receivablesTurnover', 'payablesTurnover', 'debtRatio', 'debtEquityRatio', 'priceEarningsRatio', 'priceBookValueRatio', 'dividendYield']


In [5]:
@dataclasses.dataclass
class LearnedAlgorithm:
    algorithm: str
    filename: Path
    learned_algorithm: Union[A2C, PPO, DDPG, A2C, TD3]


def get_algorithm(filename: Path):
    if "a2c" in filename.as_posix():
        return LearnedAlgorithm(algorithm="a2c", filename=filename, learned_algorithm=A2C.load(filename))


learned_algorithms = [get_algorithm(filepath) for filepath in program.exp_dir.out.algorithms.glob("*")]

In [6]:
# Env
env_kwargs = get_env_kwargs(program.dataset)
env_gym = StockTradingEnv(df=program.dataset, **env_kwargs)

Stock Dimension: 29, State Space: 494


In [7]:
# Test Learned Algorithms
df_account_value, df_actions = CustomDRLAgent.DRL_prediction(
    model=learned_algorithms[len(learned_algorithms) - 1].learned_algorithm, environment=env_gym
)

hit end!


In [8]:
df_account_value

,date,account_value
0,2019-04-12,1.000000e+06
1,2019-04-15,9.994271e+05
2,2019-04-16,1.000304e+06
3,2019-04-17,9.962704e+05
4,2019-04-18,9.988517e+05
...,...,...
923,2022-12-09,1.241070e+06
924,2022-12-12,1.260773e+06
925,2022-12-13,1.278515e+06
926,2022-12-14,1.262068e+06


In [9]:
# Get results
perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv(program.exp_dir.out.results.joinpath(f"perf_stats_all_{algorithm_name}_{now_time()}.csv"))

Annual return          0.057717
Cumulative returns     0.229539
Annual volatility      0.295385
Sharpe ratio           0.339129
Calmar ratio           0.130704
Stability              0.362444
Max drawdown          -0.441589
Omega ratio            1.068750
Sortino ratio          0.469728
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.037811
Daily value at risk   -0.036818
dtype: float64


In [10]:
start = program.dataset["date"].min()
end = program.dataset["date"].max()
print(start, end)

2019-04-12 2022-12-15


In [18]:
# Baseline
baseline_tic = yf.Ticker("^DJI")

In [19]:
baseline_df = baseline_tic.history(interval="1d", start=start, end=end)
# baseline_df.insert(0, "date", baseline_df.index)
# baseline_df.reset_index(drop=True, inplace=True)
baseline_df
# baseline_df = get_baseline("^DJI", start, end, "1d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-04-12 00:00:00-04:00,26357.789062,26436.679688,26309.720703,26412.300781,369910000,0,0
2019-04-15 00:00:00-04:00,26407.759766,26424.849609,26316.419922,26384.769531,250620000,0,0
2019-04-16 00:00:00-04:00,26482.189453,26530.710938,26397.189453,26452.660156,302340000,0,0
2019-04-17 00:00:00-04:00,26468.529297,26501.019531,26391.859375,26449.539062,367780000,0,0
2019-04-18 00:00:00-04:00,26463.369141,26602.419922,26444.529297,26559.539062,332850000,0,0
...,...,...,...,...,...,...,...
2022-12-08 00:00:00-05:00,33695.968750,33899.171875,33642.460938,33781.480469,271830000,0,0
2022-12-09 00:00:00-05:00,33746.710938,33844.628906,33462.648438,33476.460938,287630000,0,0
2022-12-12 00:00:00-05:00,33519.500000,34005.039062,33519.500000,34005.039062,313440000,0,0


In [33]:
date_format = "%Y"
baseline_df.index = baseline_df.index.tz_convert("UTC")
pd.to_datetime(baseline_df.index, format=date_format)

DatetimeIndex(['2019-04-12 04:00:00+00:00', '2019-04-15 04:00:00+00:00',
               '2019-04-16 04:00:00+00:00', '2019-04-17 04:00:00+00:00',
               '2019-04-18 04:00:00+00:00', '2019-04-22 04:00:00+00:00',
               '2019-04-23 04:00:00+00:00', '2019-04-24 04:00:00+00:00',
               '2019-04-25 04:00:00+00:00', '2019-04-26 04:00:00+00:00',
               ...
               '2022-12-01 05:00:00+00:00', '2022-12-02 05:00:00+00:00',
               '2022-12-05 05:00:00+00:00', '2022-12-06 05:00:00+00:00',
               '2022-12-07 05:00:00+00:00', '2022-12-08 05:00:00+00:00',
               '2022-12-09 05:00:00+00:00', '2022-12-12 05:00:00+00:00',
               '2022-12-13 05:00:00+00:00', '2022-12-14 05:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=927, freq=None)

In [29]:
baseline_df.index.dt.strftime(date_format)

AttributeError: 'DatetimeIndex' object has no attribute 'dt'

In [16]:
baseline_df.style.format({"date": "%Y-%m-%d"})

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,%Y-%m-%d,26357.789062,26436.679688,26309.720703,26412.300781,369910000,0,0
1,%Y-%m-%d,26407.759766,26424.849609,26316.419922,26384.769531,250620000,0,0
2,%Y-%m-%d,26482.189453,26530.710938,26397.189453,26452.660156,302340000,0,0
3,%Y-%m-%d,26468.529297,26501.019531,26391.859375,26449.539062,367780000,0,0
4,%Y-%m-%d,26463.369141,26602.419922,26444.529297,26559.539062,332850000,0,0
5,%Y-%m-%d,26510.769531,26553.050781,26458.609375,26511.050781,232570000,0,0
6,%Y-%m-%d,26513.830078,26695.960938,26503.560547,26656.390625,311690000,0,0
7,%Y-%m-%d,26652.560547,26680.580078,26582.859375,26597.050781,283370000,0,0
8,%Y-%m-%d,26426.369141,26536.480469,26310.279297,26462.080078,296730000,0,0
9,%Y-%m-%d,26454.619141,26543.560547,26392.550781,26543.330078,317290000,0,0


In [17]:
baseline_df

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-04-12 00:00:00-04:00,26357.789062,26436.679688,26309.720703,26412.300781,369910000,0,0
1,2019-04-15 00:00:00-04:00,26407.759766,26424.849609,26316.419922,26384.769531,250620000,0,0
2,2019-04-16 00:00:00-04:00,26482.189453,26530.710938,26397.189453,26452.660156,302340000,0,0
3,2019-04-17 00:00:00-04:00,26468.529297,26501.019531,26391.859375,26449.539062,367780000,0,0
4,2019-04-18 00:00:00-04:00,26463.369141,26602.419922,26444.529297,26559.539062,332850000,0,0
...,...,...,...,...,...,...,...,...
922,2022-12-08 00:00:00-05:00,33695.968750,33899.171875,33642.460938,33781.480469,271830000,0,0
923,2022-12-09 00:00:00-05:00,33746.710938,33844.628906,33462.648438,33476.460938,287630000,0,0
924,2022-12-12 00:00:00-05:00,33519.500000,34005.039062,33519.500000,34005.039062,313440000,0,0
925,2022-12-13 00:00:00-05:00,34268.441406,34712.281250,33890.878906,34108.640625,442650000,0,0


In [13]:
print(baseline_df["date"].min())
print(baseline_df["date"].max())

2019-04-12 00:00:00-04:00
2022-12-14 00:00:00-05:00


In [14]:
stats = backtest_stats(baseline_df, value_col_name="Close")

Annual return          0.070772
Cumulative returns     0.286005
Annual volatility      0.232728
Sharpe ratio           0.411239
Calmar ratio           0.190831
Stability              0.561379
Max drawdown          -0.370862
Omega ratio            1.087791
Sortino ratio          0.567932
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.953450
Daily value at risk   -0.028941
dtype: float64


In [22]:
backtest_plot(
    df_account_value,
    baseline_ticker="^DJI",
    baseline_start=program.dataset["date"].min(),
    baseline_end=program.dataset["date"].max(),
)

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (927, 8)


Start date,2019-04-12
End date,2022-12-15
Total months,44
,Backtest
Annual return,5.772%
Cumulative returns,22.954%
Annual volatility,29.539%
Sharpe ratio,0.34
Calmar ratio,0.13
Stability,0.36
Max drawdown,-44.159%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,44.16,2020-02-12,2020-03-20,2021-01-06,236
1,32.13,2021-11-02,2022-09-27,NaT,NaN
2,9.42,2019-07-23,2019-08-14,2019-09-11,37
3,7.11,2019-04-30,2019-06-03,2019-06-20,38
4,6.81,2021-08-27,2021-09-21,2021-10-18,37


Stress Events,mean,min,max
New Normal,0.04%,-15.71%,13.63%
